In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็น
!pip install -q transformers[torch] torch accelerate scikit-learn

import pandas as pd
import numpy as np
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.7 MB/s eta 0:00:00
อุปกรณ์ที่ใช้: cuda


In [22]:
# %% 2. การโหลดและเตรียมข้อมูล (CORD Dataset)

print("กำลังโหลดและประมวลผลข้อมูล CORD...")
# โหลดข้อมูล CORD จาก Hugging Face Hub (ใช้เฉพาะส่วน train มาสาธิต)
raw_dataset = load_dataset("naver-clova-ix/cord-v2", split="train")

def create_example_from_cord(example):
    """
    ฟังก์ชันสำหรับแปลง 1 ตัวอย่างข้อมูลจาก CORD ให้มี invoice_text และ ground_truth_json
    """
    ground_truth = json.loads(example["ground_truth"])

    # --- ส่วนที่แก้ไข ---
    # สร้าง invoice_text โดยการวนลูปเข้าไปในแต่ละบรรทัด และแต่ละคำ
    all_words = []
    # ตรวจสอบว่า 'valid_line' มีอยู่จริงและไม่เป็น None
    if ground_truth.get("valid_line"):
        for line in ground_truth["valid_line"]:
            # ตรวจสอบว่า 'words' มีอยู่จริงและไม่เป็น None
            if line.get("words"):
                for word_info in line["words"]:
                    all_words.append(word_info["text"])

    full_text = " ".join(all_words)
    # --- สิ้นสุดส่วนที่แก้ไข ---

    # สกัดข้อมูลเป้าหมายจาก meta data
    company_name = ground_truth['meta'].get('store_name')
    invoice_date = ground_truth['meta'].get('payment_date')
    total_amount = ground_truth['meta'].get('total_price')

    # สร้าง JSON ground truth สำหรับ few-shot example
    ground_truth_dict = {
        "company_name": company_name,
        "invoice_date": invoice_date,
        "total_amount": total_amount
    }

    return {
        "invoice_text": full_text,
        "ground_truth_json": json.dumps(ground_truth_dict, ensure_ascii=False)
    }

# ประมวลผลและสร้าง DataFrame
processed_dataset = raw_dataset.map(create_example_from_cord, remove_columns=raw_dataset.column_names)
df = processed_dataset.to_pandas().dropna().sample(frac=1, random_state=42).reset_index(drop=True)


# --- คัดเลือกตัวอย่างคุณภาพสูงสำหรับ Few-Shot Prompt ---
few_shot_example_1 = df.iloc[0]
few_shot_example_2 = df.iloc[1]

# --- สร้างข้อมูลสำหรับ Test Set เพื่อจำลองการแข่งขัน ---
test_df = df.iloc[10:20].copy() # ลดขนาดเพื่อความรวดเร็วในการทดลอง
test_df['invoice_id'] = [f"inv_{i:03d}" for i in range(len(test_df))]

print("คัดเลือกตัวอย่าง Few-shot และเตรียม Test set สำเร็จ")
print("\n--- ตัวอย่าง Few-shot 1 ---")
print(f"Text: {few_shot_example_1['invoice_text'][:]}...")
print(f"JSON: {few_shot_example_1['ground_truth_json']}")


กำลังโหลดและประมวลผลข้อมูล CORD...


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

คัดเลือกตัวอย่าง Few-shot และเตรียม Test set สำเร็จ

--- ตัวอย่าง Few-shot 1 ---
Text: 1 Spaghetti Deluxe 8.182 1 Lychee Float 5.909 1 SB 1 HC 27.727 1 Perkedel 5.909 Sub Total 47,727 P.Rest 10 % 4.773 Total 52,500 Cash 60,000 Kembali 7,500 4 Items,...
JSON: {"company_name": null, "invoice_date": null, "total_amount": null}


In [19]:
# %% 3. การโหลดโมเดล LLM และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"
print(f"กำลังโหลด Tokenizer และโมเดล LLM: '{model_id}'...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # ใช้ bfloat16 เพื่อลดการใช้ VRAM
    device_map="auto"           # โหลดโมเดลข้าม GPU อัตโนมัติ
)
model.eval() # ตั้งเป็น evaluation mode

print("โหลดโมเดล LLM สำเร็จ")


กำลังโหลด Tokenizer และโมเดล LLM: 'Qwen/Qwen2-7B-Instruct'...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

โหลดโมเดล LLM สำเร็จ


In [23]:
# %% 4. การออกแบบ Prompt (Few-Shot & JSON Output)

def create_extraction_prompt(invoice_text):
    """
    สร้าง Prompt ที่สมบูรณ์แบบ Few-shot และสั่งให้ตอบเป็น JSON
    """
    chat_template = [
        # 1. System Message: กำหนดบทบาท, คำสั่งหลัก, และกฎ
        {
            "role": "system",
            "content": """You are an expert data extraction assistant. Your task is to extract specific information from an invoice text.
You must respond ONLY with a valid JSON object containing the following keys: 'company_name', 'invoice_date', 'total_amount'.
If a piece of information cannot be found in the text, you must use the value null."""
        },

        # 2. Few-shot Example 1
        {
            "role": "user",
            "content": f"Extract information from the following invoice text:\n\n---\n{few_shot_example_1['invoice_text']}\n---"
        },
        {
            "role": "assistant",
            "content": few_shot_example_1['ground_truth_json']
        },

        # 3. Few-shot Example 2
        {
            "role": "user",
            "content": f"Extract information from the following invoice text:\n\n---\n{few_shot_example_2['invoice_text']}\n---"
        },
        {
            "role": "assistant",
            "content": few_shot_example_2['ground_truth_json']
        },

        # 4. User Query (ข่าวที่เราต้องการวิเคราะห์)
        {
            "role": "user",
            "content": f"Extract information from the following invoice text:\n\n---\n{invoice_text}\n---"
        }
    ]
    return chat_template

# ทดสอบสร้าง Prompt
test_prompt = create_extraction_prompt("This is a test invoice.")
print("สร้าง Template Prompt สำเร็จ")
# print(json.dumps(test_prompt, indent=2)) # สามารถ uncomment เพื่อดูโครงสร้างเต็มๆ ได้


สร้าง Template Prompt สำเร็จ


In [24]:
# %% 5. การทำนายผลด้วย LLM และการ Parse JSON

results = []
default_json = {"company_name": None, "invoice_date": None, "total_amount": None}

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Extracting with LLM"):
    # สร้าง Prompt สำหรับ invoice แต่ละใบ
    prompt = create_extraction_prompt(row['invoice_text'])

    # แปลง Prompt เป็น Token IDs และส่งเข้า GPU
    inputs = tokenizer.apply_chat_template(
        prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(DEVICE)

    # จำกัดความยาว Input เพื่อป้องกัน Error
    if inputs.shape[1] > 4096:
        inputs = inputs[:, -4096:]

    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=128,         # จำกัดความยาวของคำตอบให้พอดีกับ JSON
            do_sample=False,            # ไม่ต้องสุ่มคำตอบเพื่อให้ผลลัพธ์คงที่
            pad_token_id=tokenizer.eos_token_id
        )

    # ถอดรหัสคำตอบ
    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # พยายาม Parse JSON จากคำตอบอย่างรัดกุม
    try:
        # ค้นหา JSON object แรกที่เจอใน response
        start_index = response_text.find('{')
        end_index = response_text.rfind('}') + 1
        json_part = response_text[start_index:end_index]
        parsed_json = json.loads(json_part)
        # ตรวจสอบว่า key ที่จำเป็นอยู่ครบหรือไม่
        for key in default_json.keys():
            if key not in parsed_json:
                parsed_json[key] = None
        results.append(parsed_json)
    except (json.JSONDecodeError, AttributeError, ValueError):
        results.append(default_json.copy()) # หากเกิดข้อผิดพลาดในการ parse ให้ใช้ค่า default

print("\nสกัดข้อมูลด้วย LLM สำเร็จ")


Extracting with LLM:   0%|          | 0/10 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'


สกัดข้อมูลด้วย LLM สำเร็จ


In [25]:
# %% 6. การสร้างไฟล์ Submission

# สร้าง DataFrame จาก list ของผลลัพธ์
submission_df = pd.DataFrame(results)

# ตรวจสอบและจัดเรียงคอลัมน์ให้ตรงตามโจทย์
submission_df = submission_df[['company_name', 'invoice_date', 'total_amount']]

# แทรก invoice_id จาก test_df เข้าไปเป็นคอลัมน์แรก
submission_df.insert(0, 'invoice_id', test_df['invoice_id'].values)

print("\nตัวอย่างข้อมูลในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission_llm_extraction.csv", index=False)

print("\nสร้างไฟล์ submission_llm_extraction.csv สำเร็จ!")

# (ทางเลือก) ดูผลลัพธ์เทียบกับ Ground Truth
print("\n--- เปรียบเทียบผลลัพธ์กับ Ground Truth (เพื่อการประเมิน) ---")
comparison_df = test_df[['invoice_id']].copy()
comparison_df['ground_truth'] = test_df['ground_truth_json'].apply(json.loads)
comparison_df['prediction'] = results
print(comparison_df.head())



ตัวอย่างข้อมูลในไฟล์ Submission:
  invoice_id company_name invoice_date total_amount
0    inv_000         None         None       650100
1    inv_001         None         None       14.000
2    inv_002         None         None       111000
3    inv_003         None         None         65.0
4    inv_004         None         None        39600

สร้างไฟล์ submission_llm_extraction.csv สำเร็จ!

--- เปรียบเทียบผลลัพธ์กับ Ground Truth (เพื่อการประเมิน) ---
   invoice_id                                       ground_truth  \
10    inv_000  {'company_name': None, 'invoice_date': None, '...   
11    inv_001  {'company_name': None, 'invoice_date': None, '...   
12    inv_002  {'company_name': None, 'invoice_date': None, '...   
13    inv_003  {'company_name': None, 'invoice_date': None, '...   
14    inv_004  {'company_name': None, 'invoice_date': None, '...   

                                           prediction  
10  {'company_name': None, 'invoice_date': None, '...  
11  {'company_name': N

In [26]:
results

[{'company_name': None, 'invoice_date': None, 'total_amount': 650100},
 {'company_name': None, 'invoice_date': None, 'total_amount': '14.000'},
 {'company_name': None, 'invoice_date': None, 'total_amount': 111000},
 {'company_name': None, 'invoice_date': None, 'total_amount': 65.0},
 {'company_name': None, 'invoice_date': None, 'total_amount': 39600},
 {'company_name': None, 'invoice_date': None, 'total_amount': 155.999},
 {'company_name': None, 'invoice_date': None, 'total_amount': '48.000'},
 {'company_name': None, 'invoice_date': None, 'total_amount': 40000},
 {'company_name': None, 'invoice_date': None, 'total_amount': 135500},
 {'company_name': None, 'invoice_date': None, 'total_amount': '14.000'}]